In [ ]:
from PIL import Image
import cv2
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import os, time
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import math
import datetime

height, width = 120,120
resize = (height, width)

def capture_and_release(fname, resize=resize, num_frames=None, show=False):
    cap = cv2.VideoCapture(fname)
    rets, frames = [], []
    i = 0
    while True:
        ret, frame = cap.read()
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        
        if not ret:
            break
        rets.append(ret)
        frame = np.array(Image.fromarray(frame.astype(np.uint8)).resize(resize, 
                                    Image.ANTIALIAS)) / 255.0
        frames.append(frame)
        i += 1
        if num_frames is not None and i == num_frames:
            break

    if show:
        f1 = frames[1]
        cv_rgb = cv2.cvtColor(f1, cv2.COLOR_BGR2RGB)
        print "Shape: ", cv_rgb.shape
        print "Number of frames: ", len(frames)
        plt.imshow(cv_rgb)
        plt.show()

    cap.release()
    cv2.destroyAllWindows()
    return frames

label_hash = {
    0: "boxing", 1: "handclapping", 2: "handwaving", 3: "jogging", 4: "running",
    5: "walking"
}

num_labels = 6

video_files = [os.listdir(label_hash[i]) for i in range(num_labels)]

In [ ]:
all_frames = [[capture_and_release(label_hash[i] + "/" + f, resize=resize, num_frames=50)
                         for f in video_files[i]] for i in range(num_labels)]

for i in range(num_labels):
    X = []
    for frames in all_frames[i]:
        X = X + frames
    all_frames[i] = X
labels = [[i for frame in all_frames[i]] for i in range(num_labels)]

In [ ]:
print len(all_frames)
print len(labels)
print len(all_frames[0])
print len(labels[0])

In [ ]:
print all_frames[0][100].shape
plt.imshow(all_frames[0][100])

In [ ]:
X = []
for frames in all_frames:
    X = X + frames
# del all_frames
y = []
for label in labels:
    y = y + label
# del labels

X = np.array(X)
y = np.array(y)
print X.shape
print y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [ ]:
def autoencoder(input_shape=[None, height, width, 3],
                n_filters=[3, 30, 50, 50],
                filter_sizes=[5, 10, 10, 10],
                corruption=False):
    # input to the network
    x = tf.placeholder(tf.float32, input_shape, name='x')

    # ensure 2-d is converted to square tensor.
    if len(x.get_shape()) == 2:
        x_dim = np.sqrt(x.get_shape().as_list()[1])
        if x_dim != int(x_dim):
            raise ValueError('Unsupported input dimensions')
        x_dim = int(x_dim)
        x_tensor = tf.reshape(
            x, [-1, x_dim, x_dim, n_filters[0]])
    elif len(x.get_shape()) == 4:
        x_tensor = x
    else:
        raise ValueError('Unsupported input dimensions')
    current_input = x_tensor

    # Build the encoder
    encoder_weights = []
    encoder_ops = []
    shapes = []
    for layer_i, n_output in enumerate(n_filters[1:]):
        n_input = current_input.get_shape().as_list()[3]
        shapes.append(current_input.get_shape().as_list())
        W = tf.Variable(
            tf.random_uniform([
                filter_sizes[layer_i],
                filter_sizes[layer_i],
                n_input, n_output],
                -1.0 / math.sqrt(n_input),
                1.0 / math.sqrt(n_input)))
        b = tf.Variable(tf.zeros([n_output]))
        encoder_weights.append(W)
        output = tf.nn.sigmoid(
#         output = tf.nn.relu6(
            tf.add(tf.nn.conv2d(
                current_input, W, strides=[1, 1, 1, 1], padding='SAME'), b))
        encoder_ops.append(output)
        current_input = output

    # store the latent representation
    z = current_input
    encoder_weights.reverse()
    shapes.reverse()

    # Build the decoder using the same weights
    for layer_i, shape in enumerate(shapes):
        W = encoder_weights[layer_i]
        b = tf.Variable(tf.zeros([W.get_shape().as_list()[2]]))
        output = tf.nn.sigmoid(
#         output = tf.nn.relu6(
            tf.add(tf.nn.conv2d_transpose(
                current_input, W,
                tf.pack([tf.shape(x)[0], shape[1], shape[2], shape[3]]),
                strides=[1, 1, 1, 1], padding='SAME'), b))
        current_input = output
        
    decoder = current_input

    # now have the reconstruction through the network
    y = current_input
    # cost function measures pixel-wise difference
    cost = tf.reduce_sum(tf.square(y - x_tensor))

    return {'x': x, 'z': z, 'y': y, 'cost': cost, 
            "encoder": encoder_ops, "decoder": decoder}


def test_kth(X_train, X_test, n_filters, filter_sizes):
    import tensorflow as tf
    ae = autoencoder(n_filters=n_filters, filter_sizes=filter_sizes)

    learning_rate = 0.01
    optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(ae['cost'])

    # We create a session to use the graph
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    sess.run(tf.initialize_all_variables())

    # Fit all training data
    batch_size = 100
    n_epochs = 50
    step_size = 10
    for epoch_i in range(n_epochs):
        for batch_i in range(X_train.shape[0] // batch_size):
            batch_xs = X_train[batch_i * batch_size:(batch_i + 1) * batch_size]
            train = batch_xs
            sess.run(optimizer, feed_dict={ae['x']: train})
        if epoch_i % step_size == 0:
            print(str(datetime.datetime.now()), epoch_i, sess.run(ae['cost'], feed_dict={ae['x']: train}))

    # Plot example reconstructions
    n_examples = 10
    test_xs = X_test[:n_examples]
    test_xs_norm = test_xs
    recon = sess.run(ae['y'], feed_dict={ae['x']: test_xs_norm})
    print(recon.shape)
    fig, axs = plt.subplots(2, n_examples, figsize=(10, 2))
    for example_i in range(n_examples):
        axs[0][example_i].imshow(
            np.reshape(test_xs[example_i, :], (height, width, 3)))
        axs[0][example_i].axis("off")

        axs[1][example_i].imshow(
            np.reshape(
                np.reshape(recon[example_i, ...], (height, width, 3)),
                (height, width, 3)))
        axs[1][example_i].axis("off")
    
    fig.show()
    plt.draw()
    
    ae["session"] = sess
    
    return ae

In [ ]:
ae = test_kth(X_train, X_test,
                n_filters=[3, 3, 3, 3],
                filter_sizes=[3, 3, 3])

In [ ]:
combined = []
sess = ae["session"]
batch_size = 100
for batch_i in range(X_train.shape[0] // batch_size):
    batch_xs = X_train[batch_i * batch_size:(batch_i + 1) * batch_size]
    layers = [sess.run(ae["encoder"][i], 
            feed_dict={ae['x']: batch_xs}) for i in range(len(ae["encoder"]))]
    ravels = (np.array([row.ravel() for row in layers[i]]) for i in range(len(ae["encoder"])))
    interm = np.hstack((ravels))
    combined.append(interm)

combined = np.vstack((combined))[:5000]
print combined.shape
del ae
del X_train
del sess

# Confusion matrix

In [ ]:
y = y_train[:combined.shape[0]]

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(combined, y)
print str(datetime.datetime.now())

In [ ]:
print knn.score(combined, y)
print str(datetime.datetime.now())

In [ ]:
y_pred = knn.predict(combined)
print str(datetime.datetime.now())

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import numpy as np

plt.figure(figsize=(30, 30))

label_hash = {
    0: "boxing", 1: "handclapping", 2: "handwaving", 3: "jogging", 4: "running",
    5: "walking"
}
class_names = [val for key,val in label_hash.items()]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, str(cm[i, j])[:4],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

cnf_matrix = confusion_matrix(y_train[:combined.shape[0]][:1000], y_pred[:1000])
np.set_printoptions(precision=2)

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix for KTH')

plt.show()

In [ ]:
# Scale and visualize the embedding vectors
def plot_embedding(X, y, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure(figsize=(20, 10))
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 12})

    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)

vectorized_imgs = combined

from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
X_tsne = tsne.fit_transform(vectorized_imgs)

In [ ]:
plot_embedding(X_tsne, y_train)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
# knn.fit(X_tsne, y_train[:3000])
knn.fit(X_tsne, y_train[:X_tsne.shape[0]])

In [ ]:
# knn.score(X_tsne, y_train[:3000])
knn.score(X_tsne, y_train[:X_tsne.shape[0]])

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
# knn.fit(combined, y_train[:3000])
knn.fit(combined, y_train[:combined.shape[0]])

In [ ]:
# knn.score(combined, y_train[:3000])
knn.score(combined, y_train[:combined.shape[0]])

# Just k-nn

In [ ]:
X_train_ravelled = np.array([row.ravel() for row in X_train])
X_train = X_train_ravelled

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

In [ ]:
knn.score(X_train, y_train)

# Just k-means

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6, random_state=0)
kmeans.fit(X_train)

def cluster_acc(Y_pred, Y):
    """
    Finds the cluster accuracy
    """
    from sklearn.utils.linear_assignment_ import linear_assignment
    assert Y_pred.size == Y.size
    D = max(Y_pred.max(), Y.max())+1
    w = np.zeros((D,D), dtype=np.int64)
    for i in xrange(Y_pred.size):
        w[Y_pred[i], Y[i]] += 1
    ind = linear_assignment(w.max() - w)
    return sum([w[i,j] for i,j in ind])*1.0/Y_pred.size, w

y_pred = kmeans.predict(X_train)
cluster_acc(y_pred, y_train)

# Just t-SNE

In [ ]:
tsne = TSNE(n_components=2)
np.set_printoptions(suppress=True)
X_tsne = tsne.fit_transform(X_train)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_tsne, y_train)

In [ ]:
knn.score(X_tsne, y_train[:X_tsne.shape[0]])